In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trendyol-coderspace-datathon/user_demographics.csv
/kaggle/input/trendyol-coderspace-datathon/products.csv
/kaggle/input/trendyol-coderspace-datathon/qa.csv
/kaggle/input/trendyol-coderspace-datathon/supplier_defective_return.csv
/kaggle/input/trendyol-coderspace-datathon/reviews.csv
/kaggle/input/trendyol-coderspace-datathon/supplier_disputed_return.csv
/kaggle/input/trendyol-coderspace-datathon/test_data.csv
/kaggle/input/trendyol-coderspace-datathon/supplier_return.csv
/kaggle/input/trendyol-coderspace-datathon/transactions.csv
/kaggle/input/notebook1a7d790594/__results__.html
/kaggle/input/notebook1a7d790594/__notebook_source__.ipynb
/kaggle/input/notebook1a7d790594/trendyol.db
/kaggle/input/notebook1a7d790594/__notebook__.ipynb
/kaggle/input/notebook1a7d790594/__output__.json
/kaggle/input/notebook1a7d790594/custom.css


In [3]:
import nltk
from nltk.corpus import stopwords
import spacy
from textblob import TextBlob
from textblob import Word
from datetime import datetime
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

2021-10-06 07:02:10.685320: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-06 07:02:10.685431: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
pd.set_option('float_format', '{:f}'.format)

In [6]:
trans = pd.read_csv('../input/trendyol-coderspace-datathon/transactions.csv')
qa = pd.read_csv('../input/trendyol-coderspace-datathon/qa.csv')
reviews = pd.read_csv('../input/trendyol-coderspace-datathon/reviews.csv')
demog = pd.read_csv('../input/trendyol-coderspace-datathon/user_demographics.csv')
defective = pd.read_csv('../input/trendyol-coderspace-datathon/supplier_defective_return.csv')
disputed = pd.read_csv('../input/trendyol-coderspace-datathon/supplier_disputed_return.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
products =  pd.read_csv('../input/trendyol-coderspace-datathon/products.csv')

In [8]:
test = pd.read_csv('../input/trendyol-coderspace-datathon/test_data.csv')

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7471 entries, 0 to 7470
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        7471 non-null   object 
 1   expected  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 116.9+ KB


In [15]:
trans.isna().sum()

order_date                      0
user_id                         0
is_elite_user                   0
supplier_id                     0
order_line_item_id              0
order_parent_id                 0
product_content_id              0
product_variant_id              0
original_price                  0
discounted_price                0
ship_cost                   36886
coupon_id                10433735
coupon_discount                 0
promotion_name            5653910
promotion_award_value     5653910
is_wallet_trx                   0
is_saved_card_trx               0
is_returned                421016
dtype: int64

In [23]:
test.head(10)

,id,expected
0,18115|87073974|726348863,NaN
1,27588|118753298|723169934,NaN
2,29165|35699988|724122443,NaN
3,29969|5893187|722342150,NaN
4,35107|93897418|723421057,NaN
5,39921|113319776|726194743,NaN
6,65368|35624085|723622573,NaN
7,77885|104775220|729545122,NaN
8,77885|113319776|723231008,NaN
9,80646|103711355|727444257,NaN


In [30]:
7471/(421016+7471)

0.01743576817966473

In [27]:
trans[(trans['order_parent_id'] == 727444257)]

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned
11039345,2021-08-05 17:02:44.458000+00:00,80646,1,968,1117098697,727444257,38423377,68177950,79.990000,63.990000,1.651667,NaN,0.000000,NaN,NaN,0,True,NaN
11039346,2021-08-05 17:02:44.458000+00:00,80646,1,968,1117098696,727444257,90323075,150991697,69.990000,41.990000,1.651667,NaN,0.000000,TRENDYOLMİLLA & TRENDYOL MAN & TRENDYOL MODEST...,40.000000,0,True,NaN
11039347,2021-08-05 17:02:44.458000+00:00,80646,1,968,1117098695,727444257,98637439,164141002,44.990000,35.990000,1.651667,NaN,0.000000,NaN,NaN,0,True,NaN
11039348,2021-08-05 17:02:44.458000+00:00,80646,1,968,1117098699,727444257,39432459,69798335,89.990000,76.490000,1.651667,NaN,0.000000,NaN,NaN,0,True,NaN
11039349,2021-08-05 17:02:44.458000+00:00,80646,1,968,1117098700,727444257,103711355,171340663,99.990000,79.990000,1.651667,NaN,0.000000,NaN,NaN,0,True,NaN
11039350,2021-08-05 17:02:44.458000+00:00,80646,1,968,1117098698,727444257,88048779,147219201,99.990000,84.990000,1.651667,NaN,0.000000,NaN,NaN,0,True,NaN


In [42]:
id_list = []

test[['user_id','product_content_id', 'order_parent_id']] = test.id.str.split('|', expand = True,)

In [52]:
test [['user_id', 'product_content_id', 'order_parent_id']] = test[['user_id', 'product_content_id', 'order_parent_id']].astype('int')

In [53]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7471 entries, 0 to 7470
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  7471 non-null   object 
 1   expected            0 non-null      float64
 2   user_id             7471 non-null   int64  
 3   product_content_id  7471 non-null   int64  
 4   order_parent_id     7471 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 292.0+ KB


In [49]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11193677 entries, 0 to 11193676
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   order_date             object 
 1   user_id                int64  
 2   is_elite_user          int64  
 3   supplier_id            int64  
 4   order_line_item_id     int64  
 5   order_parent_id        int64  
 6   product_content_id     int64  
 7   product_variant_id     int64  
 8   original_price         float64
 9   discounted_price       float64
 10  ship_cost              float64
 11  coupon_id              object 
 12  coupon_discount        float64
 13  promotion_name         object 
 14  promotion_award_value  float64
 15  is_wallet_trx          int64  
 16  is_saved_card_trx      bool   
 17  is_returned            float64
dtypes: bool(1), float64(6), int64(8), object(3)
memory usage: 1.4+ GB


In [55]:
test_full = trans.merge(test, left_on = ['user_id', 'product_content_id', 'order_parent_id'], right_on =  ['user_id', 'product_content_id', 'order_parent_id'], how='inner' )

In [56]:
test_full.to_csv('test_full.csv')

In [45]:
trans[trans['user_id'].isin(test['user_id'])]

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned


In [10]:
trans.describe()

,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_discount,promotion_award_value,is_wallet_trx,is_returned
count,11193677.000000,11193677.000000,11193677.000000,11193677.000000,11193677.000000,11193677.000000,11193677.000000,11193677.000000,11193677.000000,11156791.000000,11193677.000000,5539767.000000,11193677.000000,10772661.000000
mean,21589352.957083,0.321320,65346.712365,1008799150.784485,659003941.388699,70947265.779872,121177778.463901,99.684125,67.459838,5.791371,-0.938158,27.769379,0.411435,0.127205
std,16315675.078293,0.466983,103280.020034,62030671.379292,38807373.992399,30946803.878870,48275601.962600,102.396744,65.068097,3.441158,6.333608,16.657705,0.492094,0.333203
min,2141.000000,0.000000,62.000000,900623500.000000,595758968.000000,48771.000000,1010649.000000,0.000000,0.000000,0.177600,-599.000000,0.100000,0.000000,0.000000
25%,10755931.000000,0.000000,968.000000,953998126.000000,624699593.000000,41679153.000000,74664632.000000,49.990000,34.990000,2.960000,0.000000,15.000000,0.000000,0.000000
50%,16489160.000000,0.000000,968.000000,1009282259.000000,655886924.000000,80712984.000000,137492817.000000,69.990000,50.990000,4.700000,0.000000,21.000000,0.000000,0.000000
75%,29422721.000000,1.000000,110969.000000,1061361159.000000,694507579.000000,95029375.000000,158889681.000000,119.990000,79.990000,9.400000,0.000000,35.000000,1.000000,0.000000
max,71173209.000000,1.000000,434994.000000,1149210041.000000,729966388.000000,137904554.000000,224372804.000000,12220.000000,6950.000000,13.200000,0.020000,2000.000000,1.000000,1.000000


In [11]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11193677 entries, 0 to 11193676
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   order_date             object 
 1   user_id                int64  
 2   is_elite_user          int64  
 3   supplier_id            int64  
 4   order_line_item_id     int64  
 5   order_parent_id        int64  
 6   product_content_id     int64  
 7   product_variant_id     int64  
 8   original_price         float64
 9   discounted_price       float64
 10  ship_cost              float64
 11  coupon_id              object 
 12  coupon_discount        float64
 13  promotion_name         object 
 14  promotion_award_value  float64
 15  is_wallet_trx          int64  
 16  is_saved_card_trx      bool   
 17  is_returned            float64
dtypes: bool(1), float64(6), int64(8), object(3)
memory usage: 1.4+ GB


In [12]:
trans.isna().sum()

order_date                      0
user_id                         0
is_elite_user                   0
supplier_id                     0
order_line_item_id              0
order_parent_id                 0
product_content_id              0
product_variant_id              0
original_price                  0
discounted_price                0
ship_cost                   36886
coupon_id                10433735
coupon_discount                 0
promotion_name            5653910
promotion_award_value     5653910
is_wallet_trx                   0
is_saved_card_trx               0
is_returned                421016
dtype: int64

In [13]:
trans.head()

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned
0,2021-05-01 00:00:01.761000+00:00,3941815,0,4662,901280147,595758968,95503167,159147197,79.900000,51.940000,8.880000,NaN,0.000000,Sepette %35 İndirim,35.000000,0,True,0.000000
1,2021-05-01 00:00:01.761000+00:00,3941815,0,163,901280149,595758968,36313798,64622938,129.950000,59.970000,9.400000,NaN,0.000000,Sepette %40 İndirim,40.000000,0,True,0.000000
2,2021-05-01 00:00:01.761000+00:00,3941815,0,157306,901280145,595758968,36888320,65612791,59.990000,49.990000,10.670000,NaN,0.000000,NaN,NaN,0,True,0.000000
3,2021-05-01 00:00:01.761000+00:00,3941815,0,163856,901280144,595758968,36888399,65613254,49.900000,49.900000,10.670000,NaN,0.000000,NaN,NaN,0,True,0.000000
4,2021-05-01 00:00:02.742000+00:00,15996105,1,968,901279297,595758987,96930601,161408176,89.990000,71.990000,9.400000,NaN,0.000000,Sepette %20 İndirim,20.000000,0,False,0.000000


In [14]:
trans[trans['is_returned']==1.000000]

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned
9,2021-05-01 00:00:03.895000+00:00,11914979,1,968,901280113,595759022,79859300,135389165,99.990000,62.990000,1.342857,NaN,0.000000,NaN,NaN,0,True,1.000000
26,2021-05-01 00:00:05.771000+00:00,12990383,0,968,901280305,595759086,37210836,66164256,149.990000,119.990000,9.400000,NaN,0.000000,NaN,NaN,0,True,1.000000
47,2021-05-01 00:00:10.354000+00:00,3713121,1,107671,901280752,595759258,91657791,152987568,179.990000,68.490000,8.880000,NaN,0.000000,NaN,NaN,1,False,1.000000
48,2021-05-01 00:00:11.517000+00:00,47716137,0,968,901280566,595759306,82247311,138929632,99.990000,48.440000,3.133333,NaN,0.000000,NaN,NaN,0,True,1.000000
49,2021-05-01 00:00:11.517000+00:00,47716137,0,968,901280565,595759306,83169613,140217818,109.990000,56.090000,3.133333,NaN,0.000000,NaN,NaN,0,True,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10772637,2021-07-31 23:59:44.733000+00:00,11540266,0,968,1108222427,722306176,64953967,112839713,129.990000,110.490000,0.979579,NaN,0.000000,NaN,NaN,1,False,1.000000
10772638,2021-07-31 23:59:44.733000+00:00,11540266,0,968,1108222419,722306176,73294399,126230753,89.990000,62.990000,0.979579,NaN,0.000000,NaN,NaN,1,False,1.000000
10772640,2021-07-31 23:59:44.733000+00:00,11540266,0,968,1108222425,722306176,73294400,126230758,89.990000,76.490000,0.979579,NaN,0.000000,NaN,NaN,1,False,1.000000
10772650,2021-07-31 23:59:49.564000+00:00,14784148,1,4442,1108222903,722306224,6482077,20914907,79.930000,39.950000,8.880000,NaN,0.000000,NaN,NaN,0,False,1.000000


In [15]:
products.head()

,product_id,product_content_id,product_variant_id,product_name,brand_id,brand_name,gender_id,gender_name,category_id,category_name,color_id,color_name,supplier_color_name,attributet_name,attribute_value
0,86747186,49156804,87148849,3'lü Müslin Örtü Seti - %100 Pamuk Bebek Müsli...,995186,Li Puna,NaN,Unknown,4979,Müslin Bez,NaN,NaN,UNKNOWN,BEDEN,Tek Ebat
1,37159399,44391553,78625630,Haşema Nehar Bakır Çizgili Boru Paça Siyah Tes...,35046,Remsa Mayo,4295.000000,Kadın / Kız,2760,Tesettür Mayo,7005.000000,Metalik,BAKIR,BEDEN,XL
2,83771873,96708156,161092495,Zincir Fantezi Bel Zinciri-0610,1074023,vonda wear,4295.000000,Kadın / Kız,3098,Fantezi Aksesuar,6996.000000,Altın,Gold,BEDEN,Tek Ebat
3,66208922,77520098,132019116,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,XS
4,68189697,79713829,135202630,Ibıza Uzun Tunik,1021561,Jadeandmate,4295.000000,Kadın / Kız,1018,Plaj Elbisesi,6996.000000,Altın,Multi,BEDEN,M/L


In [16]:
products.describe()

,product_id,product_content_id,product_variant_id,brand_id,gender_id,category_id,color_id
count,1457567.000000,1457567.000000,1457567.000000,1457567.000000,1455115.000000,1457567.000000,1456040.000000
mean,62792967.794007,74588454.044851,126459481.238599,332421.196760,4304.716772,940.957449,7005.030816
std,29012842.021660,32529128.800768,50457843.957274,470178.275718,370.210681,724.458414,5.171519
min,45197.000000,48771.000000,1010649.000000,12.000000,4294.000000,391.000000,6996.000000
25%,35843350.500000,44611560.500000,79919320.500000,908.000000,4294.000000,563.000000,6999.000000
50%,70790535.000000,84759071.000000,143256878.000000,13424.000000,4295.000000,601.000000,7004.000000
75%,85742720.000000,99997861.500000,166374562.500000,962958.000000,4295.000000,1182.000000,7009.000000
max,117817166.000000,137904554.000000,224372804.000000,1278260.000000,18088.000000,5029.000000,7015.000000


In [17]:
print('Number of unique brands:', products.brand_id.nunique() ) 
print('Number of unique gender name:', products.gender_id.nunique() ) 
print('Number of unique category:', products.category_id.nunique() ) 
print('Number of unique color name:', products.color_name.nunique() ) 
print('Number of unique supplier color name:', products.supplier_color_name.nunique() ) 
print('Number of unique attribute value:', products.attribute_value.nunique() ) 



print('unique color name:', products.color_name.unique() ) 
print('unique supplier color name:', products.supplier_color_name.unique() ) 
print('unique attribute value:', products.attribute_value.unique() ) 

Number of unique brands: 22677
Number of unique gender name: 4
Number of unique category: 197
Number of unique color name: 20
Number of unique supplier color name: 43982
Number of unique attribute value: 1217
unique color name: [nan 'Metalik' 'Altın' 'Gümüş' 'Bej' 'Beyaz' 'Gri' 'Kahverengi' 'Kırmızı'
 'Lacivert' 'Mavi' 'Mor' 'Pembe' 'Sarı' 'Siyah' 'Turkuaz' 'Turuncu'
 'Yeşil' 'Bordo' 'Ekru' 'Haki']
unique supplier color name: ['UNKNOWN' 'BAKIR' 'Gold' ... 'haki1' 'HAKİ7' 'haki-6']
unique attribute value: ['Tek Ebat' 'XL' 'XS' ... '3XL/LNG' 'M Plus' 'XL-R']


In [18]:
products.attribute_value.value_counts().head(50)

M            172385
L            160284
S            147037
XL           108526
38            56810
40            52771
36            49254
42            43748
XXL           36105
XS            35928
Tek Ebat      28131
34            27242
44            23229
3XL           16102
46            14748
7-8 Yaş       13789
9-10 Yaş      13702
5-6 Yaş       13323
3-4 Yaş       12880
11-12 Yaş     12839
2XL           12754
4-5 Yaş       12309
S/M           10991
48            10773
6-7 Yaş       10674
12-18 AY       8261
9-12 Ay        7470
50             7394
6-9 AY         7353
3 YAŞ          6705
3-6 AY         6657
18-24 AY       6585
2 YAŞ          6546
13-14 Yaş      6533
4 YAŞ          6256
L/XL           6166
2-3 Yaş        6152
8-9 Yaş        6061
10-11 Yaş      6017
6 YAŞ          5963
32             5766
4XL            5746
5 YAŞ          5700
8 Yaş          5459
0-3 AY         5042
12-13 Yaş      4981
52             4833
10 Yaş         4394
7 Yaş          4392
M/L            4365


In [19]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457567 entries, 0 to 1457566
Data columns (total 15 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   product_id           1457567 non-null  int64  
 1   product_content_id   1457567 non-null  int64  
 2   product_variant_id   1457567 non-null  int64  
 3   product_name         1457567 non-null  object 
 4   brand_id             1457567 non-null  int64  
 5   brand_name           1457567 non-null  object 
 6   gender_id            1455115 non-null  float64
 7   gender_name          1457567 non-null  object 
 8   category_id          1457567 non-null  int64  
 9   category_name        1457567 non-null  object 
 10  color_id             1456040 non-null  float64
 11  color_name           1456040 non-null  object 
 12  supplier_color_name  1457566 non-null  object 
 13  attributet_name      1457567 non-null  object 
 14  attribute_value      1457567 non-null  object 
dty

In [20]:
products.isna().sum()

product_id                0
product_content_id        0
product_variant_id        0
product_name              0
brand_id                  0
brand_name                0
gender_id              2452
gender_name               0
category_id               0
category_name             0
color_id               1527
color_name             1527
supplier_color_name       1
attributet_name           0
attribute_value           0
dtype: int64

In [4]:
import sqlite3

In [5]:
connection = sqlite3.connect('../input/notebook1a7d790594/trendyol.db')
cursor = connection.cursor()

In [33]:
trans.to_sql('transactions', connection, if_exists='append', index=False)

In [53]:
cursor.execute("CREATE TABLE disputed (supplier_id,\
total_claim,\
unresolved_claim,\
unresolved_accepted_claim,\
unresolvedclaim_percentage);")
disputed.to_sql('disputed', connection, if_exists='append', index=False)

In [5]:
for x in cursor.execute("SELECT * FROM transactions LIMIT 10"):
    print(x)

('2021-05-01 00:00:01.761000+00:00', 3941815, 0, 4662, 901280147, 595758968, 95503167, 159147197, 79.9, 51.94, 8.88, None, 0.0, 'Sepette %35 İndirim', 35.0, 0, 1, 0.0)
('2021-05-01 00:00:01.761000+00:00', 3941815, 0, 163, 901280149, 595758968, 36313798, 64622938, 129.95, 59.97, 9.4, None, 0.0, 'Sepette %40 İndirim', 40.0, 0, 1, 0.0)
('2021-05-01 00:00:01.761000+00:00', 3941815, 0, 157306, 901280145, 595758968, 36888320, 65612791, 59.99, 49.99, 10.67, None, 0.0, None, None, 0, 1, 0.0)
('2021-05-01 00:00:01.761000+00:00', 3941815, 0, 163856, 901280144, 595758968, 36888399, 65613254, 49.9, 49.9, 10.67, None, 0.0, None, None, 0, 1, 0.0)
('2021-05-01 00:00:02.742000+00:00', 15996105, 1, 968, 901279297, 595758987, 96930601, 161408176, 89.99, 71.99, 9.4, None, 0.0, 'Sepette %20 İndirim', 20.0, 0, 0, 0.0)
('2021-05-01 00:00:03.895000+00:00', 11914979, 1, 968, 901280119, 595759022, 85975599, 143946056, 59.99, 32.29, 1.3428571428571432, None, 0.0, None, None, 0, 1, 0.0)
('2021-05-01 00:00:03.895

In [ ]:
import pandas as pd
pd.read_sql_query("SELECT * FROM transactions t JOIN products p ON t.product_content_id = p.product_content_id", connection).to_csv('trans_product.csv')



In [2]:
db_df

NameError: name 'db_df' is not defined

In [ ]:
#products_trans = trans.merge(products, left_on='product_content_id', right_on = 'product_content_id', how='inner')

In [2]:
products_trans

NameError: name 'products_trans' is not defined

In [14]:
qa.head()

,product_content_id,platform,supplier_id,question,answer
0,100268920,IOS,215473,İstek üzerine bir kutu içinde 2.3. Ve 5. Sırad...,"Merhaba, maalesef şu an için talebinizi karşıl..."
1,91760306,IOS,132710,Beden tablosu paylaşabilir misiniz?,"Merhaba, beden tablosu aşağıdaki gibidir. Kend..."
2,102152800,IOS,299298,"1.90 Boy, 88 kiloyum. Oversize için ideal boy ...",Merhaba efendim L beden sizin için uygundur
3,102286287,IOS,202978,boy 173 kilo 62 hangi beden olmalı,"Merhabalar, S beden sizin için uygundur sorun ..."
4,66184893,IOS,204213,85 kiloyum L beden olur mu,Merhaba efendim Rahatlıkla olur


In [15]:
reviews.head()

,product_content_id,rate,comment,review_like_count,supplier_id
0,35244093,5,75c göğüse S beden aldım spor için daha da sar...,NaN,968
1,97842293,5,fiyatı yüksek ama çok memnunum,NaN,367
2,86701595,5,yorumlara ve beden tablosuna bakarak 1 beden b...,1.000000,191874
3,107237246,3,çok kısa iade ettim. sıradan geldi,NaN,968
4,45019480,5,Renkler daha canlı. Hoş,NaN,968


In [16]:
demog.head()

,user_id,birth_date,membership_date,gender
0,22013146,NaN,2019-05-09 11:11:17.413000+00:00,Bay
1,49707264,NaN,2020-12-19 18:32:16.240000+00:00,Bay
2,48146878,NaN,2020-11-28 12:26:46.870000+00:00,Bay
3,53528681,NaN,2021-02-01 13:16:24.117000+00:00,Bay
4,17523184,NaN,2018-09-30 22:07:07.623000+00:00,Bay


In [17]:
demog.isna().sum()

user_id                 0
birth_date         268727
membership_date         0
gender                  0
dtype: int64

In [18]:
demog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706439 entries, 0 to 706438
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   user_id          706439 non-null  int64 
 1   birth_date       437712 non-null  object
 2   membership_date  706439 non-null  object
 3   gender           706439 non-null  object
dtypes: int64(1), object(3)
memory usage: 21.6+ MB


In [19]:
reviews['rate'].value_counts()

5    3936834
4     881416
1     571183
3     507946
2     279462
Name: rate, dtype: int64

In [2]:
!wget http://setup.johnsnowlabs.com/kaggle.sh -O - | bash

--2021-10-05 19:40:52--  http://setup.johnsnowlabs.com/kaggle.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/kaggle.sh [following]
--2021-10-05 19:40:53--  https://setup.johnsnowlabs.com/kaggle.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/kaggle_setup.sh [following]
--2021-10-05 19:40:53--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/kaggle_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.13

In [13]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, SQLContext

from pyspark.ml import Pipeline
#from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml import PipelineModel

In [121]:
spark = SparkSession.builder\
    .appName("second_attempt")\
    .master("local[*]")\
    .config('spark.jars.packages', 'org.xerial:sqlite-jdbc:3.36.0')\
    .config("spark.driver.extraClassPath", ".local/share/sqlite-jdbc-3.36.0.jar")\
    .config("spark.executor.extraClassPath", ".local/share/sqlite-jdbc-3.36.0.jar")\
    .getOrCreate()

In [ ]:
!pyspark --conf spark.executor.extraClassPath='./sqlite-jdbc-3.36.0.jar ' --driver-class-path './sqlite-jdbc-3.36.0.jar' --jars "./sqlite-jdbc-3.36.0.jar" --master local[*]

In [124]:
df = spark.read.format('jdbc').options(url= "jdbc:sqlite:trendyol.db",
    dbtable = "transactions", driver='org.sqlite.JDBC').load()

Py4JJavaError: An error occurred while calling o275.load.
: java.lang.ClassNotFoundException: org.sqlite.JDBC
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:471)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:99)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:35)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:32)
	at o

In [90]:
!readlink -f sqlite-jdbc-3.36.0.jar

/root/.local/share/sqlite-jdbc-3.36.0.jar


In [130]:
transc = spark.read.csv('../input/trendyol-coderspace-datathon/transactions.csv', header=True, inferSchema=False)

In [131]:
transc.show(truncate=True)

+--------------------+--------+-------------+-----------+------------------+---------------+------------------+------------------+--------------+----------------+------------------+---------+---------------+--------------------+---------------------+-------------+-----------------+-----------+
|          order_date| user_id|is_elite_user|supplier_id|order_line_item_id|order_parent_id|product_content_id|product_variant_id|original_price|discounted_price|         ship_cost|coupon_id|coupon_discount|      promotion_name|promotion_award_value|is_wallet_trx|is_saved_card_trx|is_returned|
+--------------------+--------+-------------+-----------+------------------+---------------+------------------+------------------+--------------+----------------+------------------+---------+---------------+--------------------+---------------------+-------------+-----------------+-----------+
|2021-05-01 00:00:...| 3941815|            0|       4662|         901280147|      595758968|          95503167|    

In [11]:
df = spark.read.format('jdbc').\
     options(url='jdbc:sqlite:../input/notebook1a7d790594/trendyol.db',\
     dbtable='transactions',driver='org.sqlite.JDBC').load()

AttributeError: 'Builder' object has no attribute 'read'